In [ ]:
import pandas as pd
import io
import requests
import datetime
import plotly.express as px

## Get the Timeseries Data

Add documentation about the time series here.

In [ ]:
from functions import get_time_series

confirmed, deaths, recovered, time_series_dates = get_time_series(local=True)

## Get all the Daily Reports 

Add documentation about these reports here.

In [ ]:
from functions import get_daily_reports

daily_report_data, daily_report_dates = get_daily_reports(local=True)

## All Done Gathering Data.

Let's merge the time_series_data into one dataframe

In [ ]:
from model import data_by_area
from data import us_confirmed 

data = data_by_area(area='Nebraska', col='State', df=us_confirmed)
data.plot()

In [ ]:

def data_by_area(area='US', col='Country/Region', df=None):
	from data import time_series_date_list
	print(col)
	data =pd.Series(
		[df.loc[(df[col] == area)][date].sum() for date in time_series_date_list],
		)
	return data


In [ ]:
data_by_area(area='US', df=confirmed).plot()

In [ ]:
from functions import make_country_labels, make_state_labels

from data import confirmed

data = confirmed.copy()
print(data.groupby('Country/Region').sum().iloc[:,-1].sort_values(ascending=False).index)


make_country_labels(data=confirmed)

In [ ]:
data.groupby('Country/Region').sum().iloc[:,-2].sort_values(ascending=False)

In [ ]:
import pandas as pd
def get_daily_reports(local=True):
    start_date = pd.to_datetime('01/22/2020')
    end_date = pd.to_datetime('today')
    dates = pd.date_range(start_date, end_date)
    daily_reports = {}
    all_reports = []
    if local == True:
        print('Getting daily reports using local data')
        for date in dates:
            date_str = date.strftime('%m-%d-%Y')
            file_name = 'data/' + date_str + '.csv'
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    elif local == False:
        print('Getting daily reports using on-line data')
        daily_report_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
        for date in dates:
            # File format: 'MM-DD-YYYY.csv'
            date_str = date.strftime('%m-%d-%Y')
            file_name =date_str + '.csv'
            url = daily_report_url + file_name
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    daily_reports = pd.concat(all_reports, axis=0, ignore_index=True)
#    valid_dates = [pd.to_datetime(date) for date in list(daily_reports.keys())]
    return daily_reports

In [ ]:
df = get_daily_reports()

df['Date']
df.Date = pd.to_datetime(df.Date)
#df['Last Update'] = pd.to_datetime(df['Last Update'])
#df['Last Update']

In [ ]:
print(df.columns)
area = 'US'
most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta('12 hours')
df[(df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values('Last Update', ascending=False)

In [ ]:
def recent_updates(area='US', timedelta='12 hours', sort='Confirmed', df=None):
    ascending = {
        'Province/State': True
    }
    most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta(timedelta)
    return df[
        (df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values(
            sort, ascending=ascending.get(sort, False))


In [ ]:
from model import data_by_area
from data import time_series_dates, confirmed, get_states
import numpy as np

all_countries = confirmed.groupby('Country/Region').sum()

pandemic = 100
pandemic_countries = all_countries[all_countries['3/13/20'] >= pandemic].copy()

#confirmed[confirmed.columns[4:]].diff(axis=1)/confirmed[confirmed.columns[4:]].plot()

ndays = 7
data = pandemic_countries[pandemic_countries.columns[-ndays:-1]].copy()
data = data.sort_values(list(data.columns)[-1:], ascending=False)

rates = (data.diff(axis=1)/data)
rates = rates[rates.columns[1:]]
rates = rates.sort_values(list(rates.columns)[-1:], ascending=False)

from math import log
doubling_times = log(2)/rates
doubling_times[doubling_times == np.inf]=None
#print(doubling_times.mean(axis=1).sort_values())

get_states(df=confirmed)

In [3]:
from model import make_data_global

data = make_data_global()
data.diff()

,confirmed,deaths,recovered
1/22/20,NaN,NaN,NaN
1/23/20,98.0,1.0,2.0
1/24/20,288.0,8.0,6.0
1/25/20,493.0,16.0,3.0
1/26/20,684.0,14.0,13.0
1/27/20,809.0,26.0,9.0
1/28/20,2651.0,49.0,46.0
1/29/20,588.0,2.0,19.0
1/30/20,2068.0,38.0,17.0
1/31/20,1693.0,42.0,79.0
